<a href="https://colab.research.google.com/github/mbc2009/Inferno/blob/main/AISI/Soln.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/mbc2009/Inferno/blob/main/Soln.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**TOC**<a id='toc0_'></a>    
1. [Q3](#toc1_)    
1.1. [a](#toc1_1_)    
1.1.1. [Principle of Operation](#toc1_1_1_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=true
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# 1. <a id='toc1_'></a>[Q3](#toc0_)

## 1.1. <a id='toc1_1_'></a>[a](#toc0_)

<!-------- Workflow -------->
Basically, at each level, the GNN/MPNN model will iteratively excutes the following updating routine layer by layer:
$$
\begin{aligned}
\textbf{Step 1.} \, &\textbf{Message generating}\\
&\boxed{
  \mathbf{m}^{ij,L}
  \;=
  \; M_L
  \left(
    \mathbf{n}^{i,L-1},\,
    \mathbf{n}^{j,L-1},\,
    \mathbf{e}^{ij,L-1}\;\;
  \right)
}\\
&\text{
$\mathbf{m}^{ij,L}:$ Message array sending from node $i$ to node $j$ at ${L}^{\{\text{th}}$level}\\
&\text{
$\mathbf{n}^{i,L-1} \& \mathbf{n}^{j,L-1}:$  State (or attribute) array of node $i$ and node $j$ at previous ${L-1}^{\text{th}}$ level.
}\\
&\text{
$\mathbf{e}^{ij,L-1}:$ State (or attribute) arrayof edge between node $i$ and $j$, $(i,j)$ at at previous ${L-1}^{\text{th}}$ level
}\\
&\downarrow \\
\textbf{Step 2.} \, &\textbf{Node updating}\\
&\boxed{
  \mathbf{n}^{i,L}
  \;= \;
  U_L\!
  \left(
    \mathbf{n}^{i,L-1},\,
    \sum_{j \in \mathbf{N}(i)}
    \mathbf{m}^{ij,L}\;\;
  \right)
}\\
&\text{
  123
}\\
&\downarrow\\
\textbf{Step 3.} \, &\textbf{Edge updating}\\
&\boxed{
  \mathbf{e}^{ij,L}
  \;=\;
  \mathcal{N}_L
  \left(
      \mathbf{n}^{i,L},\,
      \mathbf{n}^{j,L},\,
      \mathbf{e}^{ij,L-1}
  \right)
}\\
&\text{
  123
}
\end{aligned}
$$

<!-------- Equations explaination -------->
More specifically, at ${L}^{\text{th}}$ level, GNN/MPNN model excutes the updating routine by  ultilizing the information extracted rom prevous ${L-1}^{\text{th}}$ level:

**Update Formulas and Workflow**

Given a universal GNN/MPNN structure with edge updates, the update rules at layer $L$ are defined as follows. The process iterates layer by layer, allowing local information to propagate through the graph.


1. **Message Generation**  
   The message $\mathbf{m}^{ij,L}$ from node $j$ to node $i$ is generated based on the previous layer's node features and edge features:
   $$
   \mathbf{m}^{ij,L} \;=\; M_L\bigl(\mathbf{n}^{i,L-1},\, \mathbf{n}^{j,L-1},\, \mathbf{e}^{ij,L-1}\bigr)
   $$
   - \(\mathbf{m}^{ij,L}\) is the message vector at layer \(L\).  
   - \(M_L\) is the learnable message function (often an MLP or other neural module) that takes as input the node features \(\mathbf{n}^{i,L-1}\), \(\mathbf{n}^{j,L-1}\) and the edge features \(\mathbf{e}^{ij,L-1}\).

2. **Node Update**  
   Each node \(i\) updates its feature \(\mathbf{n}^{i,L}\) by aggregating all incoming messages:
   $$
   \mathbf{n}^{i,L} \;=\; U_L\!\Bigl(\mathbf{n}^{i,L-1},\, \sum_{j \in \mathbf{N}(i)} \mathbf{m}^{ij,L}\Bigr)
   $$
   - \(\mathbf{n}^{i,L}\) is node \(i\)’s feature vector at layer \(L\).  
   - \(U_L\) is the node update function, which fuses the node’s old state \(\mathbf{n}^{i,L-1}\) with the sum of incoming messages.  
   - \(\mathbf{N}(i)\) is the set of neighbors of node \(i\).

3. **Edge Update**  
   After nodes are updated, each edge \((i, j)\) also gets updated based on the new node features and its previous edge features:
   $$
   \mathbf{e}^{ij,L} \;=\; \mathcal{N}_L\bigl(\mathbf{n}^{i,L},\, \mathbf{n}^{j,L},\, \mathbf{e}^{ij,L-1}\bigr)
   $$
   - \(\mathbf{e}^{ij,L}\) is the edge feature at layer \(L\).  
   - \(\mathcal{N}_L\) is the edge update function, which typically incorporates the newly updated node features \(\mathbf{n}^{i,L}, \mathbf{n}^{j,L}\) and the previous edge features \(\mathbf{e}^{ij,L-1}\).

### 1.1.1. <a id='toc1_1_1_'></a>[Principle of Operation](#toc0_)

- **Iterative Layers**: The three steps (message generation, node update, and edge update) repeat across multiple layers. This iterative mechanism expands the “receptive field” of each node and edge, enabling global context to be captured over several layers.  
- **Message Passing**: Each node’s feature is enriched by aggregated messages from its neighbors, ensuring local information flows through the graph.  
- **Edge Update**: By also updating edge features, the model can dynamically adjust relationships between nodes according to the latest node states, enhancing representational power.

Overall, this framework provides a flexible and powerful way to model node and edge attributes jointly, which is essential for many graph-based learning tasks.


<!-------- Notation definition -------->
The definitions of the notations employed in the given system of equations are provided below:
$$
\begin{aligned}
\mathbf{e}&: \text{Attribution array of edges}\\  
\mathbf{n}&: \text{Attribution array of nodes}\\  
\mathbf{m}&: \text{Message array}\\  
\mathbf{L}&: \text{Layer index}\\  
i,j &: \text{Node indices}\\  
\mathbf{N}&: \text{Neighborhood operator}\\  
\mathcal{N}&: \text{Edges updating function}\\  
\mathcal{U}&: \text{Nodes updating function}\\  
\mathcal{M}&: \text{Message generating function}\\
\end{aligned}
$$